In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def henon_heiles_symplectic_euler(T, V, initial_point=[0., 0., 0., 0.], t_interval=[0.,6*np.math.pi], h=6*np.math.pi/501):

    t = np.linspace(t_interval[0], t_interval[1], h)
    h = t[1]-t[0]
    
    px = np.zeros(t.shape[0])
    py = np.zeros(t.shape[0])
    x = np.zeros(t.shape[0])
    y = np.zeros(t.shape[0])

    x[0] = initial_point[0]
    y[0] = initial_point[1]
    px[0] = initial_point[2]
    py[0] = initial_point[3]
  
    def dT(px_0=tf.constant(1.0), py_0=tf.constant(1.0)):
        px_0 = tf.constant(px_0)
        py_0 = tf.constant(py_0)
        with tf.GradientTape() as g:
            g.watch(px_0)
            g.watch(py_0)
            y = T(px_0, py_0)
        return g.gradient(y, [px_0, py_0])

    def dV(x_0=tf.constant(1.0), y_0=tf.constant(1.0)):
        x_0 = tf.constant(x_0)
        y_0 = tf.constant(y_0)
        with tf.GradientTape() as g:
            g.watch(x_0)
            g.watch(y_0)
            y = V(x_0,y_0)
        return g.gradient(y, [x_0,y_0])
    for i in range(0, t.shape[0]-1):

        x[i + 1] = x[i] + h * dT(px[i], py[i])[0]
        y[i + 1] = y[i] + h * dT(px[i], py[i])[1]

        px[i + 1] = px[i] - h * dV(x[i+1], y[i+1])[0]
        py[i + 1] = py[i] - h * dV(x[i+1], y[i+1])[1]
  
    z = np.concatenate((x.reshape(-1,1), y.reshape(-1,1), px.reshape(-1,1), py.reshape(-1,1)) , axis=1)
    z = z + tf.random.normal(z.shape, mean = 0, stddev=0.005, dtype=tf.float32)
    return T, V, z, t, t_interval, initial_point, h